In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# 連接anvil平台

In [2]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 51 kB 256 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=075c5a4bced55f5c6a898c0abdf3b6a2817ab2e1fc20b70a67db18be269b0635
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [3]:
import anvil.server

anvil.server.connect("6FDUWXBN3ZZVSOZS3PPVQM6A-B3F7ANVOVBBTZV4X")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [4]:
import anvil.media
import jieba
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

def split_content(content):
  return " ".join(jieba.cut(content))

def process_contents(df):
  df['內容'] = df['內容'].apply(split_content)
  df['內容'] = df['內容'].str.replace('\r', '')
  df['內容'] = df['內容'].str.replace('\n', '')
  df['標籤'] = df['標籤'].astype('category')
  return df

#開啟模型檔和詞彙檔
'''
#風格
style_vec = joblib.load("/drive/My Drive/Colab Notebooks/vec_clf.pkl")
style_clf = joblib.load("/drive/My Drive/Colab Notebooks/clf.pkl")

#機能
func_vec = joblib.load("/drive/My Drive/Colab Notebooks/vec_clf.pkl")
func_clf = joblib.load("/drive/My Drive/Colab Notebooks/clf.pkl")
'''
#基地
site_vec = joblib.load("/drive/My Drive/Colab Notebooks/vec_site.pkl")
site_clf = joblib.load("/drive/My Drive/Colab Notebooks/site.pkl")
'''
@anvil.server.callable
def classify_style_text(file):
    with anvil.media.TempFile(file) as f:      
      test_df = pd.read_csv(f, encoding = "utf-8")
      test_df = process_contents(test_df)
      test_df['內容']

      test_bag = style_vec.transform(test_df['內容'])
      predict = style_clf.predict(test_bag)
      #print(list(predict))

    style = list(predict)

    return style

@anvil.server.callable
def classify_functions_text(file):
    with anvil.media.TempFile(file) as f:      
      test_df = pd.read_csv(f, encoding = "utf-8")
      test_df = process_contents(test_df)
      test_df['內容']

      test_bag = func_vec.transform(test_df['內容'])
      predict = func_clf.predict(test_bag)
      #print(list(predict))

    functions = list(predict)

    return functions
'''
@anvil.server.callable
def classify_site_text(file):
    with anvil.media.TempFile(file) as f:      
      test_df = pd.read_csv(f, encoding = "utf-8")
      test_df = process_contents(test_df)
      test_df['內容']

      test_bag = site_vec.transform(test_df['內容'])
      predict = site_clf.predict(test_bag)
      #print(list(predict))

    site = list(predict)

    return site

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid resul

# colab永遠掛載

In [ ]:
anvil.server.wait_forever()